In [60]:
# Importar las librerías necesarias
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time



pd.set_option('display.max_colwidth', None)
chrome_options = Options()
chrome_options.add_argument("--start-maximized")

# Configurar las opciones de Chrome

# URL del sitio web
url = "https://www.todoshowcase.com/"

# Realizar la solicitud HTTP
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Parsear el contenido HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Encontrar todos los enlaces en la página
    links = soup.find_all('a', href=True)  # Filtrar solo enlaces con atributo href
    
    # Filtrar los enlaces que contienen el formato específico
    movie_links = [link['href'] for link in links if '/showcase/pelicula?filmid=' in link['href']]
    df = pd.DataFrame(movie_links, columns=["Links"])
    df = df.drop_duplicates()
else:
    print(f"Error al acceder al sitio web. Código de estado: {response.status_code}")

In [62]:
# Mantener solo las filas que contengan la palabra 'houseid' en cualquier columna
listado_peliculas = df[df.apply(lambda row: row.astype(str).str.contains('house_id', case=False).any(), axis=1)]

In [48]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()  # Asegúrate de tener el controlador adecuado para tu navegador

test_listado = listado_peliculas.head(10)

for pelicula in test_listado["Links"]:
    driver.get(pelicula)

    # Esperar a que la página cargue completamente
    time.sleep(5)
    
    
    try:
        # Esperar a que el contenedor de días sea visible
        contenedor_dias = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "op_days"))
        )
        botones_dias = contenedor_dias.find_elements(By.CLASS_NAME, "op_day")

        # Iterar sobre los botones, evitando el primero
        for boton in botones_dias[1:]:  # Evitar el primer elemento
            try:
                # Asegurarse de que el botón sea clickeable
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable(boton))
                
                boton.click()
                time.sleep(2)  # Esperar después de hacer click
                
                # Imprimir el texto del botón
                print(boton.get_attribute("value"))
            except Exception as e:
                print(f"Error al interactuar con el botón: {e}")
                continue
    except Exception as e:
        print(f"Error al interactuar con los botones: {e}")
        continue
    
    # Primer elemento 
    try:
        primer_boton = botones_dias[0]  # Acceder al primer elemento
        primer_boton.click()
        print(primer_boton.get_attribute("value"))
    except Exception as e:
        print(f"Error al interactuar con el primer botón: {e}")
        

# Cerrar el navegador
driver.quit()

2025-09-21
2025-09-23
2025-09-17
2025-10-30
2025-10-28
2025-09-18
2025-09-19
2025-09-20
2025-09-21
2025-09-22
2025-09-23
2025-09-24
2025-09-17
2025-09-17
2025-09-18
2025-09-19


KeyboardInterrupt: 

In [ ]:
driver = webdriver.Chrome()  

test_listado = listado_peliculas.head(10)

data = []

for pelicula in test_listado["Links"]:
    driver.get(pelicula)

    time.sleep(0)
    
    try:
        contenedor_dias = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "op_days"))
        )
        botones_dias = contenedor_dias.find_elements(By.CLASS_NAME, "op_day")
        
        botones_dias = botones_dias[1:] + [botones_dias[0]]
            
        for boton in botones_dias:  
            try:
               
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable(boton))
                
                if len(botones_dias) > 1:
                    boton.click()
                else:
                    print("Solo hay una fecha")
                    
                time.sleep(0.5) 
                
                try:
                    button = driver.find_element(By.ID, "ui-accordion-op_cinemas-header-0")
                    button.click()
                    time.sleep(1)  
                except:
                    print("No se pudo encontrar o hacer clic en el botón de horarios.")
                    continue
            
                total = []
                for i in range(1, 10, 1):
                    try:
                        categorias = driver.find_elements(By.XPATH, f"//*[@id='ui-accordion-op_cinemas-panel-0']/div[{i}]")
                        categoria = [categoria.text for categoria in categorias]
                        total.append(categoria)
                    except:
                        print("No encontrado")

                resultados = [r for r in total if r]

                categorias = [resultados[i][0] for i in range(0, len(resultados), 2)]
                horarios = [resultados[i][0] for i in range(1, len(resultados), 2)]

                try:
                    name = driver.find_element(By.CLASS_NAME, "name").text
                except:
                    name = "Desconocido"
                try:
                    cine = driver.find_element(By.ID, "ui-accordion-op_cinemas-header-0").text
                except:
                    cine = "Desconocido"
                try:
                    fecha = boton.get_attribute("value")
                except:
                    fecha = "Desconocido"

                for categoria, horario in zip(categorias, horarios):
                    data.append({
                        "Cine": cine,
                        "Película": name,
                        "Categoría": categoria,
                        "Horarios": horario,
                        "Fecha": fecha
                    })
                   
            except Exception as e:
                print(f"Error al interactuar con el botón: {e}")
                continue
    except Exception as e:
        print(f"Error al interactuar con los botones: {e}")
        continue

df = pd.DataFrame(data)

def split_every_five(text):
    return ' '.join([text[i:i+5] for i in range(0, len(text), 5)])

df['Horarios'] = df['Horarios'].apply(split_every_five)

driver.quit()

Error al interactuar con los botones: list index out of range


In [57]:
df

,Cine,Película,Categoría,Horarios,Fecha
0,IMAX Theatre (Norcenter),"David Gilmour live at the Circus Maximus, Rome",IMAX-Subtitulado,19:25,2025-09-21
1,IMAX Theatre (Norcenter),"David Gilmour live at the Circus Maximus, Rome",IMAX-Subtitulado,19:25,2025-09-23
2,IMAX Theatre (Norcenter),"David Gilmour live at the Circus Maximus, Rome",IMAX-Subtitulado,19:25,2025-09-17
3,IMAX Theatre (Norcenter),Depeche Mode: M,IMAX-Subtitulado,19:50,2025-10-30
4,IMAX Theatre (Norcenter),Depeche Mode: M,IMAX-Subtitulado,19:50,2025-10-28
5,IMAX Theatre (Norcenter),Demon Slayer: Kimetsu no Yaiba Castillo Infinito,IMAX-Subtitulado,16:05 19:20,2025-09-18
6,IMAX Theatre (Norcenter),Demon Slayer: Kimetsu no Yaiba Castillo Infinito,IMAX-Subtitulado,16:05 19:20,2025-09-19
7,IMAX Theatre (Norcenter),Demon Slayer: Kimetsu no Yaiba Castillo Infinito,IMAX-Subtitulado,12:50 16:05,2025-09-20
8,IMAX Theatre (Norcenter),Demon Slayer: Kimetsu no Yaiba Castillo Infinito,IMAX-Subtitulado,12:55 16:10,2025-09-21
9,IMAX Theatre (Norcenter),Demon Slayer: Kimetsu no Yaiba Castillo Infinito,IMAX-Subtitulado,16:05 19:20,2025-09-22
